In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import simsopt
import simsopt.geo
import simsopt.field
import os
import copy
import sys

import plotly.express as px
import plotly.graph_objects as go
from scipy.io import netcdf_file
import numpy as np


Authorization required, but no authorization protocol specified

Authorization required, but no authorization protocol specified

Authorization required, but no authorization protocol specified



In [2]:
db_path = "QUASR_db/simsopt_serials/1114"
sopt_objs = []

ID = 1010
# for ID in range(10000):
fID = ID // 1000 
# sopt_objs.append(simsopt.load(f'./simsopt_serials/{fID:04}/serial{ID:07}.json'))

for filename in os.listdir(db_path):
  sopt_objs.append(simsopt.load(os.path.join(db_path, filename)))

  # simsopt.geo.plot(np.concatenate(sopt_objs[-1]))

In [3]:
df = pd.DataFrame(sopt_objs, columns=["surfaces", "coils"])
df["lcfs"] = df["surfaces"].map(lambda x: x[-1])
df["AR"] = df["lcfs"].map(lambda x: x.aspect_ratio())
df["volume"] = df["lcfs"].map(lambda x: -x.volume())
df["nfp"] = df["lcfs"].map(lambda x: x.nfp)
df["R1"] = df["lcfs"].map(lambda x: x.minor_radius())

print(len(df))
df.head()

42


,surfaces,coils,lcfs,AR,volume,nfp,R1
0,[SurfaceXYZTensorFourier1],"[Coil1, Coil2, Coil3, Coil4, Coil5, Coil6, Coi...",SurfaceXYZTensorFourier1,20.000039,0.049348,4,0.050000
1,"[SurfaceXYZTensorFourier2, SurfaceXYZTensorFou...","[Coil25, Coil26, Coil27, Coil28, Coil29, Coil3...",SurfaceXYZTensorFourier4,6.666820,0.444132,4,0.149999
2,"[SurfaceXYZTensorFourier5, SurfaceXYZTensorFou...","[Coil49, Coil50, Coil51, Coil52, Coil53, Coil5...",SurfaceXYZTensorFourier11,2.860970,2.418053,4,0.349844
3,"[SurfaceXYZTensorFourier12, SurfaceXYZTensorFo...","[Coil73, Coil74, Coil75, Coil76, Coil77, Coil7...",SurfaceXYZTensorFourier17,3.335870,1.776529,4,0.299924
4,[SurfaceXYZTensorFourier18],"[Coil97, Coil98, Coil99, Coil100, Coil101, Coi...",SurfaceXYZTensorFourier18,20.000015,0.049348,4,0.050000


In [4]:
def eval_row(row, plot=False):
  coils = row["coils"]
  nfp = row["nfp"]
  R1 = row["R1"]
  lcfs = row["lcfs"]
  computational_surface = simsopt.geo.SurfaceRZFourier.from_nphi_ntheta(nfp=nfp, nphi=64, ntheta=64, range="half period")

  # Simple torus boundary
  computational_surface.set_rc(1,0,R1*2.0)
  computational_surface.set_zs(1,0,R1*2.0)
  computational_surface.change_resolution(3,4)
  # computational_surface.set_rc(2,1,np.random.rand())
  # computational_surface.set_zs(1,2,np.random.rand())
  normal_computational = computational_surface.normal()
  xyz_computational = computational_surface.gamma()

  # Scaled computational boundary
  scale = 1.5
  lcfs.scale(scale)
  xyz_computational = lcfs.gamma()
  normal_computational = lcfs.normal()
  lcfs.scale(1.0 / scale)

  bs = simsopt.field.BiotSavart(coils)
  bs.set_points_cart(xyz_computational.reshape((-1,3)))
  B = bs.B().reshape(xyz_computational.shape)
  BdotN = np.sum(normal_computational * B, axis=-1)
  BdotN_fft = np.fft.fft2(BdotN)
  
  if plot:
    import plotly.express as px
    # px.imshow(BdotN).show()
    # px.imshow(np.abs(np.fft.fftshift(BdotN_fft))).show()
    # px.imshow(np.real(np.fft.fftshift(BdotN_fft)), title="Real component").show()
    # px.imshow(np.imag(np.fft.fftshift(BdotN_fft)), title="Imag component").show()
    plt.figure(figsize=(16,5))
    plt.subplot(131)
    plt.imshow(BdotN)
    plt.title("BdotN")
    plt.colorbar()
    plt.subplot(132)
    plt.imshow(np.real(np.fft.fftshift(BdotN_fft)))
    plt.title("fft real")
    plt.colorbar()
    plt.subplot(133)
    plt.imshow(np.imag(np.fft.fftshift(BdotN_fft)))
    plt.title("fft imag")
    plt.colorbar()
    # plt.tight_layout()
    plt.show()

  return BdotN_fft

for idx,row in df.iterrows():
  eval_row(row, False)

- Check simple computational boundaries
- Check using a simple scaled up boundary
- Is SVD independent of boundary geometry?

In [5]:
lcfs = df["lcfs"][10]

# XYZ tensor fourier -> RZ fourier
rzf = lcfs.to_RZFourier()

In [ ]:
def netcdf_from_surface(surface:simsopt.geo.SurfaceRZFourier):
  import os

  filename_out = "wout_surfaces_python_generated.nc"
  filename = "../bdistrib/equilibria/wout_w7x_standardConfig.nc"
  os.system(f'cp {filename} {filename_out}')  

  # Copy the file on disk with a new name, open with r+ and overwrite it.
  with netcdf_file(filename_out, "a", mmap=False) as f:
    print(list(f.variables.keys()))
    print(f.variables["zmns"][()].shape, f.variables["zmns"].units, f.variables["zmns"].dimensions,  f.variables["zmns"][()].dtype)
    print(np.max(f.variables["zmns"][()]))

    # The plasma surface read in by bdistrib is zmns[ns] & rmnc[ns]
    # nfp_vmec = nfp
    # Rmajor_p = R0 

    # implicitly broadcasts the result throughout all flux surfaces
    mpol = int(f.variables["mpol"][()])-1
    ntor = int(f.variables["ntor"][()])
    rzf.change_resolution(mpol, ntor)
    f.variables["rmnc"][:] = surface.rc.flatten()[rzf.ntor:]
    f.variables["zmns"][:] = -surface.zs.flatten()[rzf.ntor:] # Right and left handed coordinates. Flip the signs of sin(theta) to match

    f.variables["Rmajor_p"][()] = surface.major_radius() 
    f.variables["nfp"][()] = surface.nfp 

    # TODO: set lmns component? sinmn component of lambda, half mesh
    # TODO: GMNC component? Both dont seem to have an impact
    # f.variables["gmnc"][:] = np.zeros(f.variables["gmnc"][()].shape)
    # f.variables["lmns"][:] = np.zeros(f.variables["lmns"][()].shape)

    # HACK sets the success flag to true so the input reading doesnt fail
    f.variables["ier_flag"][()] = 0
  
  return filename_out.replace("wout_","")

def write_bdistribin(netcdffilename, mpol = 12, ntor = 12):
  nu = 64
  nv = 256

  sep_middle = 0.05
  sep_outer = 0.1

  bdistribin = f"""&bdistrib
    transfer_matrix_option = 1

    nu_plasma={nu}
    nu_middle={nu}
    nu_outer ={nu}

    nv_plasma={nv}
    nv_middle={nv}
    nv_outer ={nv}

    ! This run is not well resolved at this resolution, but it is enough for testing.
    mpol_plasma = {mpol}
    mpol_middle = {mpol}
    mpol_outer  = {mpol}

    ntor_plasma = {ntor}
    ntor_middle = {ntor}
    ntor_outer  = {ntor}

    geometry_option_plasma = 2
    wout_filename='{netcdffilename}'

    geometry_option_middle=2
    separation_middle={sep_middle}

    geometry_option_outer=2
    separation_outer={sep_outer}

    pseudoinverse_thresholds = 1e-12

    n_singular_vectors_to_save = 16
  /
  """
  filename_out = "bdistrib_in.python_generated"
  with open(filename_out, "w") as f:
    f.write(bdistribin)
  return filename_out


import subprocess
subprocess.check_call(["../bdistrib/bdistrib", write_bdistribin(netcdf_from_surface(rzf))])

# rzf.plot()
import bdistribPlot
bdistribPlot.main("bdistrib_out.python_generated.nc")

In [11]:
for ID in range(1000):
  fID = ID // 1000 
  simsopt_path = f'./QUASR_db/simsopt_serials/{fID:04}/serial{ID:07}.json'
  if os.path.exists(simsopt_path):
    soptobj = simsopt.load(simsopt_path)

    lcfs = soptobj[0][-1]
    # XYZ tensor fourier -> RZ fourier
    rzf = lcfs.to_RZFourier()
    
    bdistrib_out_path = f'./QUASR_db/bdistrib_serials/{fID:04}/bdistrib_out.{ID:07}.nc'
    subprocess.check_call(["mkdir", "-p", os.path.dirname(bdistrib_out_path)])
    subprocess.check_call(["../bdistrib/bdistrib", write_bdistribin(netcdf_from_surface(rzf))])
    # Move results to correct directory
    subprocess.check_call(["mv", "bdistrib_out.python_generated.nc", bdistrib_out_path, "-u"])
  else:
    print("Skipping", simsopt_path)

Skipping ./QUASR_db/simsopt_serials/0000/serial0000000.json
Skipping ./QUASR_db/simsopt_serials/0000/serial0000001.json
Skipping ./QUASR_db/simsopt_serials/0000/serial0000002.json
Skipping ./QUASR_db/simsopt_serials/0000/serial0000003.json
Skipping ./QUASR_db/simsopt_serials/0000/serial0000004.json
Skipping ./QUASR_db/simsopt_serials/0000/serial0000005.json
Skipping ./QUASR_db/simsopt_serials/0000/serial0000006.json
Skipping ./QUASR_db/simsopt_serials/0000/serial0000007.json
Skipping ./QUASR_db/simsopt_serials/0000/serial0000008.json
Skipping ./QUASR_db/simsopt_serials/0000/serial0000009.json
Skipping ./QUASR_db/simsopt_serials/0000/serial0000010.json
Skipping ./QUASR_db/simsopt_serials/0000/serial0000011.json
Skipping ./QUASR_db/simsopt_serials/0000/serial0000012.json
Skipping ./QUASR_db/simsopt_serials/0000/serial0000013.json
Skipping ./QUASR_db/simsopt_serials/0000/serial0000014.json
Skipping ./QUASR_db/simsopt_serials/0000/serial0000015.json
Skipping ./QUASR_db/simsopt_serials/0000